In [1]:
import pandas as pd
import numpy as np
import os
import sklearn
import lucem_illud_2020 
import networkx as nx
import matplotlib.pyplot as plt
import seaborn #Makes the plots look nice
import scipy #Some stats
import nltk #a little language code
from IPython.display import Image #for pics
import pickle
%matplotlib inline

In [2]:
namedf=pd.read_excel('/home/lkcao/content-final/cleaned.xlsx',Sheet='sheet1')
sodf=pd.read_csv("sepsoap.txt",encoding='utf-8',sep='\t')

In [48]:
newcode=[]
for i in range(0,len(namedf)):
    df0=namedf.iloc[i]
    if df0['occupation code']==2:
        newcode.append(1)
    elif df0['occupation code']==4:
        newcode.append(3)
    else:
        newcode.append(df0['occupation code'])
namedf['new occupation code']=newcode
namedf[0:5]

,v1,name,show,times,url,actor,nickname,ethnicity,gender,born,age,occupation,occupation code,residence,causeofdeath,https://vignette.wikia.nocookie.net/onelifetolive/images/e/ea/Brody_Cast_Photo.jpg/revision/latest?cb=20130328165942,new occupation code
0,0,Luke,General Hospital,22941,https://general-hospital.fandom.com/wiki/Luke_...,Anthony Geary,"Father of the Blonde One, Proud Patriarch",Irish-American,Male,1948,72,Owner of Barrett EnterprisesCo-owner of Kelly\...,2,Amsterdam[7],NaN,https://vignette.wikia.nocookie.net/general-ho...,1
1,1,Edward,General Hospital,7656,https://general-hospital.fandom.com/wiki/Edwar...,NaN,NaN,English-American,Male,2-Sep-18,94,Founder/chairman/principal shareholder of ELQ ...,1,NaN,"Old age, heart condition",https://vignette.wikia.nocookie.net/general-ho...,1
2,2,Bobbie,General Hospital,3890,https://general-hospital.fandom.com/wiki/Bobbi...,Jacklyn Zeman,Barb,Irish-American,Female,8-Aug-50,62,Surgical nurse at GHCo-owner of Kelly\'s Diner,4,Jacks Lake House657 Harbor View RoadPort Charl...,NaN,https://vignette.wikia.nocookie.net/general-ho...,3
3,3,Zander,General Hospital,5212,https://general-hospital.fandom.com/wiki/Zande...,NaN,NaN,NaN,Male,NaN,NaN,Criminal,5,NaN,Suicide by cop; shot by the SWAT team when he ...,https://vignette.wikia.nocookie.net/general-ho...,5
4,4,Scott,General Hospital,5886,https://general-hospital.fandom.com/wiki/Scott...,Kin Shriner,The Court Prince,NaN,Male,1956,64,Attorney,1,"Port Charles, New York",NaN,https://vignette.wikia.nocookie.net/general-ho...,1


In [5]:
#topic modelling prepare
newlovedf1=pd.read_csv("newlove.txt",sep='\t',encoding='utf-8')
newlovedf1['normlized']=newlovedf1['words'].apply(lambda x:lucem_illud_2020.normalizeTokens(lucem_illud_2020.word_tokenize(x.lower()))) 

In [6]:
lines=newlovedf1[['show','speaker','listener','words','normlized']]
nameinfo=namedf[['name','show','new occupation code','gender']]
a1=pd.merge(lines,nameinfo,how='left',left_on=['show','speaker'],right_on=['show','name'])
a2=pd.merge(a1,nameinfo,how='left',left_on=['show','listener'],right_on=['show','name'])

In [9]:
import sklearn
import sklearn.feature_extraction.text
import sklearn.pipeline
import sklearn.preprocessing
import sklearn.datasets
import sklearn.cluster
import sklearn.decomposition
import sklearn.metrics
import scipy 
import gensim

In [254]:
def dropMissing(wordLst, vocab):
    return [w for w in wordLst if w in vocab]
soapTFVectorizer = sklearn.feature_extraction.text.TfidfVectorizer(max_df=0.5,min_df=3, max_features=20000000,stop_words='english', norm='l2')
soapTFVects = soapTFVectorizer.fit_transform(newlovedf1['words'])
leavingwords=list(soapTFVectorizer.vocabulary_.keys())
leavingwords.remove('uh')
leavingwords.remove('oh')
leavingwords.remove('um')
leavingwords.remove('know')
leavingwords.remove('ok')
leavingwords.remove('let')
leavingwords.remove('na')
leavingwords.remove('okay')
leavingwords.remove('want')
leavingwords.remove('yes')
leavingwords.remove('wait')
leavingwords.remove('thank')
leavingwords.remove('yeah')
leavingwords.remove('hey')
leavingwords.remove('bye')
leavingwords.remove('sure')
leavingwords.remove('guy')
leavingwords.remove('look')
leavingwords.remove('right')
leavingwords.remove('listen')
leavingwords.remove('come')
leavingwords.remove('like')
for t in namedf['name']:
    try:
        leavingwords.remove(t.lower())
    except:
        continue
newlovedf1['reduced_tokens'] = newlovedf1['normlized'].apply(lambda x: dropMissing(x, leavingwords))
dictionary = gensim.corpora.Dictionary(newlovedf1['reduced_tokens'])
corpus = [dictionary.doc2bow(text) for text in newlovedf1['reduced_tokens']]
gensim.corpora.MmCorpus.serialize('soap.mm', corpus)
soap = gensim.corpora.MmCorpus('soap.mm')

In [278]:
buslda = gensim.models.ldamodel.LdaModel(corpus=soap, id2word=dictionary, num_topics=5, alpha=0.001, eta=0.9)

In [279]:
topicsDict = {}
for topicNum in range(buslda.num_topics):
    topicWords = [w for w, p in buslda.show_topic(topicNum,topn=20)]
    topicsDict['Topic_{}'.format(topicNum)] = topicWords
buswordRanksDF = pd.DataFrame(topicsDict)
buswordRanksDF

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4
0,father,tell,think,need,life
1,child,think,good,help,day
2,life,gon,mean,talk,time
3,voice,mean,say,little,heart
4,mother,way,time,think,marry
5,tell,thing,baby,tell,feel
6,son,feel,remember,care,way
7,try,time,thing,girl,think
8,away,try,little,god,live
9,family,happen,night,mean,god


In [280]:
ldaDF = pd.DataFrame({
        'show': newlovedf1['show'],
        'name' : newlovedf1['speaker'],
        'topics' : [buslda[dictionary.doc2bow(l)] for l in newlovedf1['normlized']]})
topicsProbDict = {i : [0] * len(ldaDF) for i in range(buslda.num_topics)}
for index, topicTuples in enumerate(ldaDF['topics']):
    for topicNum, prob in topicTuples:
        topicsProbDict[topicNum][index] = prob
for topicNum in range(buslda.num_topics):
    ldaDF['topic_{}'.format(topicNum)] = topicsProbDict[topicNum]
ldaDF

,show,name,topics,topic_0,topic_1,topic_2,topic_3,topic_4
0,All My Childen,Erica,"[(1, 0.6944359), (4, 0.30546412)]",0.000000,0.694436,0.000000,0.000000,0.305464
1,All My Childen,David,"[(1, 0.9990012)]",0.000000,0.999001,0.000000,0.000000,0.000000
2,All My Childen,Ryan,"[(4, 0.999871)]",0.000000,0.000000,0.000000,0.000000,0.999871
3,All My Childen,Dixie,"[(1, 0.9994289)]",0.000000,0.999429,0.000000,0.000000,0.000000
4,All My Childen,David,"[(0, 0.15707418), (1, 0.59208417), (4, 0.25075...",0.157074,0.592084,0.000000,0.000000,0.250751
5,All My Childen,Leo,"[(2, 0.52182686), (4, 0.4777984)]",0.000000,0.000000,0.521827,0.000000,0.477798
6,All My Childen,Greenlee,"[(0, 0.74811834), (1, 0.25158182)]",0.748118,0.251582,0.000000,0.000000,0.000000
7,All My Childen,J.R.,"[(1, 0.99800503)]",0.000000,0.998005,0.000000,0.000000,0.000000
8,All My Childen,Ryan,"[(4, 0.99981827)]",0.000000,0.000000,0.000000,0.000000,0.999818
9,All My Childen,Marcus,"[(1, 0.9990012)]",0.000000,0.999001,0.000000,0.000000,0.000000


In [281]:
topiclist=[]
for i in range(0,len(ldaDF)):
    targettuple=ldaDF.iloc[i]['topics']
    scores=[x[1] for x in targettuple]
    topic=[x[0] for x in targettuple]
    place=scores.index(max(scores))
    thetopic=topic[place]
    topiclist.append(thetopic)
resulttopic=pd.DataFrame({'show':ldaDF['show'],'name':ldaDF['name'],'topic':topiclist})
resulttopic

,show,name,topic
0,All My Childen,Erica,1
1,All My Childen,David,1
2,All My Childen,Ryan,4
3,All My Childen,Dixie,1
4,All My Childen,David,1
5,All My Childen,Leo,2
6,All My Childen,Greenlee,0
7,All My Childen,J.R.,1
8,All My Childen,Ryan,4
9,All My Childen,Marcus,1


In [282]:
namedfindex=namedf[['show','name','gender','new occupation code']]
topicresult=pd.merge(resulttopic,namedfindex,how='left',left_on=['show','name'],right_on=['show','name'])
topicresult[0:5]

,show,name,topic,gender,new occupation code
0,All My Childen,Erica,1,Female,1.0
1,All My Childen,David,1,Male,1.0
2,All My Childen,Ryan,4,Male,1.0
3,All My Childen,Dixie,1,Female,1.0
4,All My Childen,David,1,Male,1.0


In [283]:
tab = pd.crosstab(topicresult['new occupation code'],topicresult['topic'],normalize='index')
tab

topic,0,1,2,3,4
new occupation code,,,,,
1.0,0.142256,0.425200,0.154425,0.105900,0.172219
3.0,0.136373,0.438176,0.154613,0.106893,0.163945
5.0,0.150271,0.436591,0.141887,0.103364,0.167887


In [285]:
tab = pd.crosstab(topicresult['new occupation code'],topicresult['topic'])
tab

topic,0,1,2,3,4
new occupation code,,,,,
1.0,4629,13836,5025,3446,5604
3.0,643,2066,729,504,773
5.0,1416,4114,1337,974,1582


In [284]:
topicresult['gender'].fillna('male',inplace=True)
topicresult['gender']=topicresult['gender'].apply(lambda x: x.lower().strip())

gender,female,male,transgender (female to male)
topic,,,
0,3727,3481,36
1,11364,9796,123
2,4091,3637,41
3,2813,2520,17
4,4355,4450,55
